# Abundances


In [ ]:
from synthesizer.abundances import (
    Abundances,
    ScalingFunctions,
    plot_abundance_pattern,
    plot_multiple_abundance_patterns,
)

by default `Abundances` creates a solar abundance pattern with no depletion.

In [ ]:
a1 = Abundances()

like most `synthesizer` objects we can explore the important attributes of an object by using `print()`:

In [ ]:
print(a1)

You can access the logarithmic abundances (log10(X/H)) of an element like this:

In [ ]:
print(f"log10(O/H): {a1.total['O']:.2f}")
print(f"log10(O/H): {a1['O']:.2f}")

We can specify a different metallicity

In [ ]:
a2 = Abundances(metallicity=0.01)
print(a2)

and include alpha-enhancement:

In [ ]:
a3 = Abundances(metallicity=0.01, alpha=0.6)
print(a3)

We can print a relative solar abundance like this:

In [ ]:
print(f"[O/Fe] = {a3.solar_relative_abundance('O', ref_element='Fe'):.2f}")
print(f"[O/Fe] = {a3['[O/Fe]']:.2f}")

That, is, as expected.

... or the dust-to-metal ratio:

In [ ]:
a4 = Abundances(metallicity=0.01, dust_to_metal_ratio=0.3)
print(a4)

When the `dust_to_metal_ratio` > 0.0 total, gas, and dust abundance patterns are provided e.g.

In [ ]:
print(f'log10(C_total/H) : {a4.total["C"]:.2f}')
print(f'log10(C_gas/H) : {a4.gas["C"]:.2f}')
print(f'log10(C_dust/H) : {a4.dust["C"]:.2f}')

We can also change the abundance of any specific element (or set of elements), with the abundances of other elements rescaled self-consistently to yield the correct metallicity. 

If the abundance is a float it is the logarithmic abundance ($log_{10}(X/H)$) while if it is a string it is one of the in-built functions that scale the abundance with metallicity (e.g. the model proposed by Dopita et al. 2006). Note, combining this with a non-zero `alpha` can lead to a mild inconsistency.

Using a float:

In [ ]:
a5 = Abundances(metallicity=0.0134, abundances={"N": -4.5})
print(a5)

Using a specified function:

In [ ]:
a6 = Abundances(metallicity=0.0134, abundances={"N": "Dopita2006"})
print(a6)

We can also access the scaling functions directly:

In [ ]:
ScalingFunctions.Dopita2006.N(0.016)

These functions also include useful meta data:

In [ ]:
print(ScalingFunctions.Dopita2006.ads)
print(ScalingFunctions.Dopita2006.doi)

## Plots

There are also a helper functions for plotting one or more abundance patterns, here we plot two abundance patterns with different alpha abundances:

In [ ]:
plot_multiple_abundance_patterns(
    [a2, a3],
    labels=[r"Z=0.01", r"Z=0.01; \alpha = 0.6"],
    show=True,
    ylim=[-7.0, -3.0],
)

We can plot the abundance pattern of each component:

In [ ]:
plot_abundance_pattern(
    a4, show=True, ylim=[-7.0, -3.0], lines=["total", "gas", "dust"]
)